## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. parameter store

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

## 1. Dataset for training

In [4]:
import os

In [8]:
strSrcPath = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
strTargetPath = f"s3://{pm.get_params(key='DAT-BUCKET')}/reviews-helpfulness-pipeline/data/reviews.tsv.gz"

's3://sagemaker-ap-northeast-2-419974056037/reviews-helpfulness-pipeline/data/reviews.tsv.gz'

In [9]:
strQuery = "".join(["aws s3 cp ", strSrcPath, " ", strTargetPath])
strResponse = os.popen(strQuery).read()

### Delete bucket (if needed)

In [27]:
strBucketName = pm.get_params(key="DAT-BUCKET")
strBucketName

'sagemaker-ap-northeast-2-419974056037'

In [29]:
strQuery = "".join(["aws s3 rm --recursive ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()
strQuery = "".join(["aws s3 rb ", '"s3://', strBucketName, '"'])
strResponse = os.popen(strQuery).read()

## 2. Build a docker and Register the image to ECR

In [10]:
from utils.ecr import ecr_handler

In [11]:
ecr = ecr_handler()

In [12]:
strRepositoryName = "dat-processing-container"
strDockerDir = "./docker/"
strTag = ":latest"
strRegionName=pm.get_params(key="DAT-REGION")
strAccountId=pm.get_params(key="DAT-ACCOUNT-ID")    

In [13]:
ecr.build_docker(strDockerDir, strRepositoryName)

/home/ec2-user/SageMaker/dynamic-ab-testing
/home/ec2-user/SageMaker/dynamic-ab-testing


In [14]:
strEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountId, strRepositoryName, strTag)
pm.put_params(key="DAT-PROCESSING-ECR-URI", value=strEcrRepositoryUri, overwrite=True)

== REGISTER AN IMAGE TO ECR ==
  processing_repository_uri: 419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/dat-processing-container:latest


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



== REGISTER AN IMAGE TO ECR ==


'Store suceess'

## 3. Set code-pipeLine

In [15]:
import os
import boto3
import sagemaker
from os import path
from utils.code_pipeline import code_pipeline_handler

In [16]:
cph = code_pipeline_handler()

#### -- args for codecommit, build and pipeline

In [17]:
strRepoName = "mlops"
strReopDesc = "MLOps for Dynamic A/B Testing"
strSrcDir = os.path.join(os.getcwd(), "./mlops/")

### 3.1 code-commit

In [18]:
strCloneURL = cph.create_repository(strRepoName, strReopDesc)
cph.clone_from_url(strSrcDir, strCloneURL)
cph.initial_commit_push(strSrcDir)
#cph.delete_repository(strRepoName)

== CREATE REPO ==
  Repository name [mlops] was successfully created!!


Cloning into 'tmp'...




/home/ec2-user/SageMaker/dynamic-ab-testing
/home/ec2-user/SageMaker/dynamic-ab-testing/mlops

[master (root-commit) 9fc3a6b] Initial commit
 Committer: EC2 Default User <ec2-user@ip-172-16-95-57.ap-northeast-2.compute.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 23 files changed, 1984 insertions(+)
 create mode 100644 .ipynb_checkpoints/codebuild-buildspec-checkpoint.yml
 create mode 100644 codebuild-buildspec.yml
 create mode 100644 pipeline/.ipynb_checkpoints/mlops_pipeline-checkpoint.py
 create mode 100644 pipeline/config/.ipynb_checkpoints/config-checkpoint.ini
 create mode 100644 pipeline/config/.ipynb_checkpoints/confi

To https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/mlops
 * [new branch]      master -> master


### 3.2 CodeBuild

In [19]:
strCodeBuildPJTName="mlops-build"

In [20]:
cph.create_build_project(
    strCodeBuildPJTName=strCodeBuildPJTName,
    strAccountId=pm.get_params(key="DAT-ACCOUNT-ID"),
    strBuildServiceRoleARN=pm.get_params(key="DAT-CODE-BUILD-ROLE-ARN"),
    strRegionName=pm.get_params(key="DAT-REGION"),
    strBucketName=pm.get_params(key="DAT-BUCKET"),
)
#!aws codebuild delete-project --name "mlops-build"

Argments for CodeBuild below:
{'artifacts': {'name': 'mlops-build', 'type': 'CODEPIPELINE'},
 'badgeEnabled': False,
 'cache': {'type': 'NO_CACHE'},
 'description': 'Hello Build Project',
 'environment': {'computeType': 'BUILD_GENERAL1_SMALL',
                 'environmentVariables': [{'name': 'IMAGE_REPO_NAME',
                                           'type': 'PLAINTEXT',
                                           'value': 'iris-model'},
                                          {'name': 'IMAGE_TAG',
                                           'type': 'PLAINTEXT',
                                           'value': 'latest'},
                                          {'name': 'AWS_ACCOUNT_ID',
                                           'type': 'PLAINTEXT',
                                           'value': '419974056037'},
                                          {'name': 'AWS_DEFAULT_REGION',
                                           'type': 'PLAINTEXT',
                         

### 3.3. CodePipeline

In [21]:
pm.get_params(key="DAT-CODE-PIPELINE-ROLE-ARN")

'arn:aws:iam::419974056037:role/DAT-CodePipelineRole'

In [22]:
strCodePipelineName = "mlops-code-pipeline"

In [23]:
cph = code_pipeline_handler()
cph.create_execute_code_pipeline(
    strCodePipelineName=strCodePipelineName,
    strPipelineRoleARN=pm.get_params(key="DAT-CODE-PIPELINE-ROLE-ARN"),
    strRegionName=pm.get_params(key="DAT-REGION"),
    strBucketName=pm.get_params(key="DAT-BUCKET"),
    strRepoName=strRepoName,
    strCodeBuildPJTName=strCodeBuildPJTName,
)
#!aws codepipeline delete-pipeline --name "mlops-code-pipeline"

Couldn't find Pipeline: [mlops-code-pipeline], so, create new pipeline.
Create CodePipeline
Argments for CodeBuild below:
{'pipeline': {'artifactStore': {'location': 'sagemaker-ap-northeast-2-419974056037',
                                'type': 'S3'},
              'name': 'mlops-code-pipeline',
              'roleArn': 'arn:aws:iam::419974056037:role/DAT-CodePipelineRole',
              'stages': [{'actions': [{'actionTypeId': {'category': 'Source',
                                                        'owner': 'AWS',
                                                        'provider': 'CodeCommit',
                                                        'version': '1'},
                                       'configuration': {'BranchName': 'master',
                                                         'OutputArtifactFormat': 'CODE_ZIP',
                                                         'PollForSourceChanges': 'false',
                                                    

In [24]:
## Start pipeline
cph.start_pipeline_execution(strCodePipelineName)

Start pipeline execution for [mlops-code-pipeline]
